In [1]:
from src.data import extract_vault, holdout
import matplotlib.pyplot as plt 
import networkx as nx
import numpy as np

In [2]:
vault = extract_vault('./vaultes/brain/')

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, blueprint python to placeholder-1671408403833.jpeg
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rel_filepath      124 non-null    object        
 1   abs_filepath      124 non-null    object        
 2   file_exists       135 non-null    object        
 3   n_backlinks       135 non-null    object        
 4   n_wikilinks       114 non-null    float64       
 5   n_tags            114 non-null    float64       
 6   n_embedded_files  114 non-null    float64       
 7   modified_time     124 non-null    datetime64[ns]
 8   graph_category    135 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 10.5+ KB
None


In [3]:
train_graph, holdout_links = holdout(vault.graph, seed=42, alpha=0.2)
sum([hl in train_graph.edges for hl in holdout_links])

0

In [4]:
from src.model_base import calculate_adamic_adar, create_pairs_for_nodes, generate_recomendations

In [5]:
for hl in holdout_links:
    print(hl[0])

hashtag synonyms
plugin ideas for Obsidian
Intro to Obsidian
1 directional annotations
winget preset idea
Hypothesis review
github wikis in obsidian & interwikilinks
github wikis in obsidian & interwikilinks
build a portfolio
Obsidian improvements
Art of giving feedback
learn technical art
learn technical art


In [6]:
holdout_nodes = [hl[0] for hl in holdout_links]
pairs = create_pairs_for_nodes(nodes=holdout_nodes, graph=train_graph)
scores = calculate_adamic_adar(graph=train_graph, pair_nodes=pairs).groupby('node_from').agg(lambda x: list(x))

In [7]:
recs = generate_recomendations(scores)

In [8]:
recs

,recomendations
note_id,
1 directional annotations,"[add notes to explorer, bash on windows, Obsid..."
Art of giving feedback,"[blueprint python, Obsidian Web - Chrome exten..."
Hypothesis review,"[Obsidian, add notes to explorer, Obsidian pri..."
Intro to Obsidian,"[Obsidian, Obsidian backup, Obsidian special f..."
Obsidian improvements,"[plugin ideas for Obsidian, winget, Obsidian, ..."
build a portfolio,"[learn technical art, Goal driven learning, de..."
github wikis in obsidian & interwikilinks,"[blueprint python, Obsidian Web - Chrome exten..."
hashtag synonyms,"[redirect for notes, sliding panes, winget, ad..."
learn technical art,"[TA brings value to the team, Goal driven lear..."


In [9]:
import pandas as pd
holdout_links_df = pd.DataFrame.from_records(holdout_links, columns=['note_id', 'notes_to', 'p']).groupby('note_id').agg(lambda x: list(x))[['notes_to']]

In [10]:
holdout_links_df

,notes_to
note_id,
1 directional annotations,[save quotes n lines]
Art of giving feedback,[gallup survey]
Hypothesis review,[Obsidian]
Intro to Obsidian,[Obsidian]
Obsidian improvements,[Obsidian submodule]
build a portfolio,[generalist vs specialist in Indie vs AAA]
github wikis in obsidian & interwikilinks,"[interwikilinks plugin, wikilinks]"
hashtag synonyms,[wikilinks]
learn technical art,"[learn 2d, visualize your progress]"


In [11]:
from src.metrics import recall_n

In [13]:
recall_n(recs=recs, holdout_links=holdout_links_df, topn=10)

0.5